In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

In [ ]:
def transform_to_patient_centric(input_path, output_path=None):
    """
    Transform medical dataset to patient-centric format for CAD detection model.
    Saves output to Google Drive.

    Args:
        input_path: Path to input CSV file
        output_path: Path for output CSV file (default: '/content/drive/MyDrive/processed_patient_data.csv')
    """

    # Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
    print("✓ Google Drive mounted successfully")

    # Set default output path if not provided
    if output_path is None:
        output_path = '/content/drive/MyDrive/processed_patient_data.csv'

    # Load the dataset
    print(f"\nLoading data from {input_path}...")
    try:
        df = pd.read_csv(input_path)
        print(f"✓ Data loaded successfully")
        print(f"Initial dataset shape: {df.shape}")
    except FileNotFoundError:
        print(f"Error: File not found at {input_path}")
        return None

    # 1. Remove duplicate rows based on SUBJECT_ID
    df_clean = df.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
    print(f"After removing duplicates: {df_clean.shape}")
    print(f"Removed {len(df) - len(df_clean)} duplicate patient records")

    # 2. Convert GENDER to numeric (M=1, F=0)
    gender_mapping = {'M': 1, 'F': 0, 'Male': 1, 'Female': 0, 'male': 1, 'female': 0}
    df_clean['GENDER'] = df_clean['GENDER'].map(gender_mapping)

    # 3. Handle missing values for Age and Gender
    # Fill missing Age with median
    if df_clean['AGE'].isnull().any():
        age_median = df_clean['AGE'].median()
        df_clean['AGE'] = df_clean['AGE'].fillna(age_median)
        print(f"Filled {df_clean['AGE'].isnull().sum()} missing Age values with median: {age_median:.1f}")

    # Fill missing Gender with mode (most frequent)
    if df_clean['GENDER'].isnull().any():
        gender_mode = df_clean['GENDER'].mode()[0]
        df_clean['GENDER'] = df_clean['GENDER'].fillna(gender_mode)
        print(f"Filled {df_clean['GENDER'].isnull().sum()} missing Gender values with mode: {gender_mode}")



In [ ]:
 # 4. Extract patient-known ICD-9 conditions
    def extract_icd9_flags(icd9_string):
        """
        Extract binary flags for patient-known conditions from ICD9_CODES string.
        """
        if pd.isnull(icd9_string) or str(icd9_string).strip() == '':
            return {'diabetes': 0, 'cholesterol': 0, 'obesity': 0}

        codes = str(icd9_string).split(';')

        # Initialize flags
        flags = {
            'diabetes': 0,
            'cholesterol': 0,
            'obesity': 0
        }

        # Check each code
        for code in codes:
            code = code.strip()
            # Skip empty codes
            if not code:
                continue
            # Diabetes (codes starting with 250)
            if code.startswith('250'):
                flags['diabetes'] = 1
            # Cholesterol/Hyperlipidemia (codes starting with 272)
            elif code.startswith('272'):
                flags['cholesterol'] = 1
            # Obesity (codes starting with 278)
            elif code.startswith('278'):
                flags['obesity'] = 1

        return flags

    # Apply the extraction function
    icd9_flags = df_clean['ICD9_CODES'].apply(extract_icd9_flags)

    # Create new columns from the extracted flags
    df_clean['is_diabetic'] = [flags['diabetes'] for flags in icd9_flags]
    df_clean['has_high_cholesterol'] = [flags['cholesterol'] for flags in icd9_flags]
    df_clean['is_obese'] = [flags['obesity'] for flags in icd9_flags]

In [ ]:
    # 5. Create CAD_LABEL (1 if codes start with 414 or 410, otherwise 0)
    def create_cad_label(icd9_string):
        """
        Create CAD label based on ICD9 codes starting with 414 or 410.
        """
        if pd.isnull(icd9_string) or str(icd9_string).strip() == '':
            return 0

        codes = str(icd9_string).split(';')

        for code in codes:
            code = code.strip()
            if code.startswith('414') or code.startswith('410'):
                return 1

        return 0

    df_clean['CAD_LABEL'] = df_clean['ICD9_CODES'].apply(create_cad_label)

In [ ]:
 # 6. Select and order the required columns
    # Define PPG columns (check which ones exist in the dataset)
    ppg_prefixes = ['ppg_min_']
    available_ppg = [col for col in df_clean.columns if any(col.startswith(prefix) for prefix in ppg_prefixes)]

    if available_ppg:
        print(f"Found {len(available_ppg)} PPG columns: {available_ppg}")
    else:
        print("Warning: No PPG columns found in the dataset")
        # Try specific columns if generic search doesn't work
        specific_ppg = ['ppg_min_3', 'ppg_min_4', 'ppg_min_5', 'ppg_min_6', 'ppg_min_7', 'ppg_min_8']
        available_ppg = [col for col in specific_ppg if col in df_clean.columns]

    # Define output columns
    output_columns = ['SUBJECT_ID', 'AGE', 'GENDER',
                      'is_diabetic', 'has_high_cholesterol', 'is_obese'] + \
                     available_ppg + ['CAD_LABEL']

    # Create final DataFrame
    processed_df = df_clean[output_columns].copy()


In [ ]:
    # 7. Ensure exactly 1,278 unique patient records
    target_count = 1278

    if len(processed_df) > target_count:
        # If we have more than target, take first target_count records
        processed_df = processed_df.head(target_count)
        print(f"Dataset truncated to {target_count} records (as requested)")
    elif len(processed_df) < target_count:
        print(f"Dataset has {len(processed_df)} unique records (target: {target_count})")
        print("Note: If you need exactly 1278 records, you may need to combine multiple datasets")

In [ ]:

    # 8. Save to Google Drive
    print(f"\nSaving processed data to Google Drive...")
    print(f"Output path: {output_path}")

    # Create directory if it doesn't exist
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created directory: {output_dir}")

    processed_df.to_csv(output_path, index=False)
    print(f"✓ Successfully saved to Google Drive")





In [ ]:
    # 9. Print summary statistics
    print("\n" + "="*60)
    print("TRANSFORMATION COMPLETE - SUMMARY REPORT")
    print("="*60)
    print(f"Output file: {output_path}")
    print(f"File size: {os.path.getsize(output_path) / 1024:.2f} KB")
    print(f"Final dataset shape: {processed_df.shape}")
    print(f"Unique patients: {processed_df['SUBJECT_ID'].nunique()}")

    print("\n📊 DEMOGRAPHICS:")
    print(f"- Age range: {processed_df['AGE'].min():.1f} - {processed_df['AGE'].max():.1f} years")
    print(f"- Mean age: {processed_df['AGE'].mean():.1f} ± {processed_df['AGE'].std():.1f} years")

    gender_counts = processed_df['GENDER'].value_counts()
    if len(gender_counts) == 2:
        print(f"- Gender: {gender_counts[1]} Male, {gender_counts[0]} Female")
        print(f"- Male/Female ratio: {gender_counts[1]/gender_counts[0]:.2f}")

    print("\n🏥 PATIENT-KNOWN CONDITIONS:")
    conditions = ['is_diabetic', 'has_high_cholesterol', 'is_obese']
    for condition in conditions:
        count = processed_df[condition].sum()
        percentage = (count / len(processed_df)) * 100
        condition_name = condition.replace('_', ' ').title()
        print(f"- {condition_name}: {count} patients ({percentage:.1f}%)")

    print("\n🎯 CAD DETECTION LABELS:")
    cad_count = processed_df['CAD_LABEL'].sum()
    cad_percentage = (cad_count / len(processed_df)) * 100
    print(f"- CAD Positive: {cad_count} patients ({cad_percentage:.1f}%)")
    print(f"- CAD Negative: {len(processed_df) - cad_count} patients ({100 - cad_percentage:.1f}%)")

    print("\n📈 DATA QUALITY CHECK:")
    print(f"- Missing values in AGE: {processed_df['AGE'].isnull().sum()}")
    print(f"- Missing values in GENDER: {processed_df['GENDER'].isnull().sum()}")
    print(f"- Missing values in CAD_LABEL: {processed_df['CAD_LABEL'].isnull().sum()}")

In [ ]:
 # Preview the data
    print("\n👁️  DATA PREVIEW (first 5 rows):")
    print(processed_df.head())

    print("\n📋 COLUMNS IN OUTPUT FILE:")
    for i, col in enumerate(processed_df.columns, 1):
        print(f"  {i:2d}. {col}")

    return processed_df, output_path





In [ ]:
def verify_google_drive_file(file_path):
    """
    Verify that the file was saved correctly to Google Drive.
    """
    print("\n" + "="*60)
    print("GOOGLE DRIVE VERIFICATION")
    print("="*60)

    try:
        if os.path.exists(file_path):
            df_check = pd.read_csv(file_path)
            print(f"✓ File exists at: {file_path}")
            print(f"✓ File contains {len(df_check)} rows and {len(df_check.columns)} columns")
            print(f"✓ First few rows loaded successfully")
            print("\nSample data from saved file:")
            print(df_check.head(3))

            # Check critical columns
            required_cols = ['SUBJECT_ID', 'AGE', 'GENDER', 'CAD_LABEL']
            missing = [col for col in required_cols if col not in df_check.columns]
            if missing:
                print(f"\n⚠️  Warning: Missing required columns: {missing}")
            else:
                print(f"\n✅ All critical columns present")

        else:
            print(f"✗ File not found at: {file_path}")

    except Exception as e:
        print(f"✗ Error verifying file: {e}")



In [ ]:
# Main execution function for Google Colab
def main():
    """
    Main function to run in Google Colab environment.
    """


    choice = input("\nEnter '1', '2', or '3': ").strip()

    if choice == '1':
        # File in Google Drive
        input_path = input("Enter full Google Drive path to your CSV file: ").strip()
    elif choice == '2':
        # File uploaded to Colab
        input_path = input("Enter path to your uploaded CSV file: ").strip()
    else:
        print("Invalid choice. Using default sample data.")
        return

        # Ask for output file name
    output_name = input("\nEnter output file name (or press Enter for default): ").strip()
    if not output_name:
        output_path = '/content/drive/MyDrive/processed_patient_data.csv'
    else:
        if not output_name.endswith('.csv'):
            output_name += '.csv'
        output_path = f'/content/drive/MyDrive/{output_name}'

    # Process the data
    try:
        processed_df, final_output_path = transform_to_patient_centric(input_path, output_path)

        # Verify the saved file
        verify_google_drive_file(final_output_path)
        print("✅ TRANSFORMATION SUCCESSFUL!")

    except Exception as e:
        print(f"\n❌ Error during processing: {e}")



In [ ]:
# Run the main function
if __name__ == "__main__":
    main()